# Simulating and Mitigating Qubit Crosstalk for an isolated 3-qubit system
This notebook analyzes a three-qubit system to demonstrate the effects of crosstalk during a gate operation and its mitigation using a compensation pulse.

## Part 1: Simulating Qubit Crosstalk without Compensation

In this section, we simulate a standard scenario where a drive pulse on a target qubit induces unwanted interactions on its neighbors.

### 1.1 System and Pulse Parameters
We define a system of $N=3$ qubits. The key physical parameters are the qubit frequency $\omega_q$, the Rabi frequency $\Omega$ of the drive pulse on the target qubit, and the inherent crosstalk coupling strength $J$. The pulse itself is a Gaussian envelope centered at $t_0$ with a temporal width of $\tau$.

In [1]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D  # for 3D Bloch


In [6]:
# --- 1. SYSTEM & PULSE DEFINITIONS ---
N = 3
target = 1
w_q     = 5.0 * 2*np.pi    # qubit freq (rad/ns)
Omega   = 10.0 * 2*np.pi   # main drive Rabi
J       = 0.1 * Omega      # inherent crosstalk coupling

t_total = 100.0
t0, tau = 50.0, 5.0

def drive_env(t, args):
    return np.exp(- (t-args['t0'])**2 / (2*args['tau']**2))

# Initial state: all qubits in the ground state |000>
psi0  = qt.tensor(*[qt.basis(2,0) for _ in range(N)])
# Time list for the simulation
tlist = np.linspace(0, t_total, 201)
# Arguments for the time-dependent pulse envelopes
args  = {'t0': t0, 'tau': tau}



We also design a visualization helper function to generate the Bloch sphere animation from the simulation results.

In [14]:
from IPython.display import Image, display 

def create_bloch_animation(result_obj, title_suffix, filename_suffix,
                           main_pulse_vals, comp_pulse_vals=None, show_comp_pulse=False):
    """
    Generates and saves a Bloch sphere animation from QuTiP simulation results,
    and displays it in the Jupyter Notebook.
    """
    times = result_obj.times
    
    # Compute Bloch expectations
    exp_x = [qt.expect(sx[i], result_obj.states) for i in range(N)]
    exp_y = [qt.expect(sy[i], result_obj.states) for i in range(N)]
    exp_z = [qt.expect(sz[i], result_obj.states) for i in range(N)]

    fig = plt.figure(figsize=(12,8))
    
    # Top panel: pulses
    ax_p = fig.add_subplot(2, 1, 1)
    ax_p.plot(times, main_pulse_vals, label='Main Drive Pulse', color='tab:blue', lw=3)
    ax_p.fill_between(times, 0, main_pulse_vals, color='tab:blue', alpha=0.1)
    
    if show_comp_pulse and comp_pulse_vals is not None:
        ax_p.plot(times, comp_pulse_vals, '--', label='Compensation Pulse', color='tab:orange', lw=3)
        ax_p.fill_between(times, 0, comp_pulse_vals, color='tab:orange', alpha=0.1)
    
    ax_p.set_ylabel('Normalized Amplitude', fontsize=12)
    ax_p.set_xlim(0, t_total)
    ax_p.set_title(f"Pulse Sequence and Qubit Evolution {title_suffix}", fontsize=14)
    ax_p.legend(loc='upper right', fontsize=11)
    pulse_marker = ax_p.axvline(0, color='k', lw=2)

    # Bottom panels: Bloch spheres
    axes, colors, arts = [], ['green','brown','blue'], []
    for i in range(N):
        ax = fig.add_subplot(2, N, N + i + 1, projection='3d')
        ax.set_box_aspect((1,1,1))
        ax.set_title(f"Qubit {i}")
        ax.set_xlim(-1,1); ax.set_ylim(-1,1); ax.set_zlim(-1,1)
        ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
        
        # Draw Bloch sphere wireframe
        u, v = np.linspace(0,2*np.pi,60), np.linspace(0, np.pi,30)
        x_sphere = np.outer(np.cos(u), np.sin(v))
        y_sphere = np.outer(np.sin(u), np.sin(v))
        z_sphere = np.outer(np.ones_like(u), np.cos(v))
        ax.plot_wireframe(x_sphere, y_sphere, z_sphere, color='lightgray', alpha=0.3)
        axes.append(ax)

    def update(k):
        pulse_marker.set_xdata([times[k], times[k]])
        
        for a in arts: a.remove()
        arts.clear()
        
        for i, ax in enumerate(axes):
            x, y, z = exp_x[i][k], exp_y[i][k], exp_z[i][k]
            a = ax.quiver(0,0,0, x,y,z, color=colors[i], lw=2.5, arrow_length_ratio=0.15)
            arts.append(a)
        return [pulse_marker] + arts

    ani = FuncAnimation(fig, update, frames=len(times), blit=False, interval=50)
    
    gif_filename = f'crosstalk_{filename_suffix}_anim.gif'
    print(f"Saving animation to {gif_filename}...")
    ani.save(gif_filename, writer='pillow', fps=20)
    print("GIF saved successfully.")
    plt.close(fig)
    
    print(f"Displaying {gif_filename} in notebook:")
    display(Image(url=gif_filename))


### 1.2 The System Hamiltonian
The total Hamiltonian $H(t)$ governs the coherent evolution of the system. It is composed of a static drift term $H_0$ and a time-dependent drive term that includes both the intended control and the unwanted crosstalk.

#### Drift Hamiltonian ($H_0$)

This term describes the static energy of the three non-interacting qubits.
$$H_0 = H_{\mathrm{drift}} = -\frac{1}{2} \sum_{i=0}^{2} \omega_q \, \sigma_z^{(i)}$$
This is implemented by summing the individual $\sigma_z$ operators.

In [8]:
# Pauli operators
sx = [qt.tensor(*([qt.qeye(2)]*i + [qt.sigmax()] + [qt.qeye(2)]*(N-1-i)))
      for i in range(N)]
sy = [qt.tensor(*([qt.qeye(2)]*i + [qt.sigmay()] + [qt.qeye(2)]*(N-1-i)))
      for i in range(N)]
sz = [qt.tensor(*([qt.qeye(2)]*i + [qt.sigmaz()] + [qt.qeye(2)]*(N-1-i)))
      for i in range(N)]
sm = [qt.tensor(*([qt.qeye(2)]*i + [qt.sigmam()] + [qt.qeye(2)]*(N-1-i)))
      for i in range(N)]
H0 = sum(-0.5*w_q*sz[i] for i in range(N))


#### Drive and Crosstalk Hamiltonian

The time-dependent part of the Hamiltonian is modulated by the Gaussian envelope drive_env(t). It consists of the intended drive on the target qubit ($H_{\mathrm{drive}}$) and the resulting crosstalk interaction ($H_{\mathrm{xtlk}}$).
$$H_{\mathrm{drive}} = \Omega \, \sigma_x^{(\text{target})}$$
$$H_{\mathrm{xtlk}} = J \left( \sigma_x^{(\text{target}-1)}\sigma_x^{(\text{target})} + \sigma_x^{(\text{target})}\sigma_x^{(\text{target}+1)} \right)$$
The total time-dependent Hamiltonian for the uncompensated system is:
$$H(t) = H_0 + \left( H_{\mathrm{drive}} + H_{\mathrm{xtlk}} \right) f(t),$$
where $ f(t) $ is a time function representing the activation of pulses.

This is constructed in QuTiP as a list, where the second element is a list containing the time-dependent operator and its corresponding envelope function.

In [4]:
H_drive =  Omega   * sx[target]
H_xtlk  =  J       * (sx[target-1]*sx[target] + sx[target]*sx[target+1])

H = [
    H0,
    [H_drive + H_xtlk, drive_env]

]


#### Isolated System Evolution
Since the system is isolated, its state remains pure for all time. The evolution of the state vector $\vert \psi(t) \rangle$ is governed by the time-dependent Schrödinger equation:
$$\boxed{
i \frac{d}{dt}\vert \psi(t) \rangle = H(t) \vert \psi(t) \rangle
}$$
We initialize the sysetm in the state $\vert \psi_0 \rangle = \vert 000 \rangle$, and solve the evolution using `qutip.seevolve`. 



In [10]:
psi0  = qt.tensor(*[qt.basis(2,0) for _ in range(N)])
tlist = np.linspace(0, t_total, 201)
args  = {'t0': t0, 'tau': tau}

print("Running isolated simulation (uncompensated)...")
result_uncomp = qt.sesolve(H, psi0, tlist, args=args)
print("Done.")



Running isolated simulation (uncompensated)...
Done.


We now use the helper function to visualize the results.

In [13]:
# Pre-calculate the main pulse values for plotting
main_pulse_vals = drive_env(tlist, args)

create_bloch_animation(
    result_uncomp,
    title_suffix="without Compensation",
    filename_suffix="uncompensated",
    main_pulse_vals=main_pulse_vals,
    show_comp_pulse=False # No compensation pulse in this visualization
)


Saving animation to crosstalk_uncompensated_anim.gif...
GIF saved successfully.
Displaying crosstalk_uncompensated_anim.gif in notebook:


**Observation:**

 The animation shows the state vector of the central qubit (1) rotating significantly on its Bloch sphere, indicating the successful application of the drive. Crucially, the Bloch vectors for the neighbor qubits (0 and 2) is also be perturbed, precessing on their respective spheres and moving away from their initial position at the north pole ($\langle\sigma_z\rangle=1$). This demonstrates the unwanted effect of crosstalk. Since the system is isolated, the length of each Bloch vector will remain exactly 1, indicating a pure state.

---
## Part 2: Mitigating crosstalk using compensation pulse

Now, we introduce a compensation pulse designed to actively cancel the unwanted crosstalk interaction in real-time.
### 2.1 The Compensation Hamiltonian
The core idea is to introduce an additional drive term, $H_{\mathrm{comp}}$, that has the same operator form as the crosstalk Hamiltonian but with an opposite sign. For ideal cancellation, the compensation drive amplitude $\Omega_c$ is set equal to the crosstalk coupling strength $J$.
$$H_{\mathrm{comp}} = -\Omega_c \left( \sigma_x^{(\text{target}-1)}\sigma_x^{(\text{target})} + \sigma_x^{(\text{target})}\sigma_x^{(\text{target}+1)} \right)$$



In [15]:
# Compensation pulse amplitude is set equal to the crosstalk coupling J
Omega_c = J

# Compensation Hamiltonian term
H_comp  = -Omega_c * (sx[target-1]*sx[target] + sx[target]*sx[target+1])

# Define a separate envelope function for clarity, though it's identical to drive_env
def comp_env(t, args):
    return np.exp(- (t-args['t0'])**2 / (2*args['tau']**2))


### 2.2 The Full Hamiltonian with Compensation
The total Hamiltonian now includes the compensation term. When added to the solver, the $H_{\mathrm{xtlk}}$ and $H_{\mathrm{comp}}$ terms will ideally destructively interfere, leaving only the intended drive on the target qubit.

$$H_{\mathrm{compensated}}(t) = H_0 + \left( H_{\mathrm{drive}} + H_{\mathrm{xtlk}} \right) f(t) + H_{\mathrm{comp}} f(t),$$
where $ f(t) $ is a time function representing the activation of pulses. Since $H_{\mathrm{xtlk}} + H_{\mathrm{comp}} = 0$ when $\Omega_c = J$, the effective Hamiltonian becomes:
$$\boxed{
H_{\mathrm{compensated}}(t) = H_0 + H_{\mathrm{drive}} f(t)
}$$

In [17]:
# Total Hamiltonian for the compensated case
H_compensated = [
    H0,
    [H_drive + H_xtlk, drive_env], # Main drive and crosstalk
    [H_comp,             comp_env]   # Compensation pulse
]


We run the simulation again using sesolve with the new, compensated Hamiltonian.

In [18]:
print("Running isolated simulation (compensated)...")
result_comp = qt.sesolve(H_compensated, psi0, tlist, args=args)
print("Done.")


Running isolated simulation (compensated)...
Done.


### 2.4 Visualization of Compensated Crosstalk
Finally, we visualize the results with the compensation pulse active.

In [19]:
# Pre-calculate compensation pulse values for plotting
comp_pulse_vals = -Omega_c / Omega * main_pulse_vals # Scale for plotting relative to main pulse

create_bloch_animation(
    result_comp,
    title_suffix="with Compensation",
    filename_suffix="compensated",
    main_pulse_vals=main_pulse_vals,
    comp_pulse_vals=comp_pulse_vals,
    show_comp_pulse=True # Plot the compensation pulse in this visualization
)


Saving animation to crosstalk_compensated_anim.gif...
GIF saved successfully.
Displaying crosstalk_compensated_anim.gif in notebook:


**Observation**: The animation shows a dramatic improvement for the neighbor qubits.

- Target Qubit (1): Its evolution should be identical to the uncompensated case, as the intended drive $H_{\mathrm{control}}$ is unaffected.

- Neighbor Qubits (0 and 2): Due to the cancellation of $H_{\mathrm{xtlk}}$ by $H_{\mathrm{comp}}$, the unwanted drive on the neighbors is completely suppressed. Their state vectors should now remain perfectly stationary at the north pole of the Bloch sphere ($\langle\sigma_z\rangle=1$), demonstrating ideal crosstalk mitigation in this isolated system.